In [1]:
import pandas as pd
import ftfy
import streamlit as st
from pymongo import MongoClient
import time
from datetime import datetime
import pandas as pd
from openai import OpenAI
import streamlit.components.v1 as components
import sys
import json
from navigation import render_nav

In [2]:
# df = pd.read_csv("final_user_batches.csv", encoding="utf-8")
# df["abstract_fixed"] = df["abstract"].apply(fix_text)

In [3]:
# bad = df.loc[df["abstract_id"] == 1045, "abstract"].iloc[0]
# print(bad)
# print(bad.encode("utf-8", errors="backslashreplace"))
# print(bad.encode("latin1", errors="backslashreplace"))

In [4]:
# df = pd.read_csv("final_user_batches.csv")

In [5]:
# print(df.isna().sum())

In [6]:
# print(df.columns.tolist())

In [7]:
# df.loc[df["user_id"] == "jchan51@illinois.edu"]

In [ ]:
def get_openai_client():
    return OpenAI(api_key="")

client_openai = get_openai_client()

In [9]:
def format_sata(sata_list):
    out = []
    for q in sata_list:
        out.append(f"{q['question']}\n")
        choices = ", ".join(q["options"])
        out.append(f"Choices: {choices}.\n")
        correct = ", ".join(q["correct_answers"])
        out.append(f"Correct answer: {correct}\n\n")

    return "".join(out)

In [10]:
# conversation_text = (
#     "User: What Gamma aminobutyric acid (GABA) receptor agonists? "
#     "Assistant: Gamma aminobutyric acid (GABA) receptor agonists are substances that bind to and activate "
#     "GABA receptors in the brain. GABA is the primary inhibitory neurotransmitter in the central nervous system, "
#     "meaning it tends to reduce neuronal excitability. By activating GABA receptors, these agonists enhance the "
#     "inhibitory effect of GABA, generally leading to a calming effect on the nervous system. This action can make "
#    ? receptor agonists being studied for their potential neuroprotective effects and role in "
#     "treating acute stroke. However, the studies highlighted seem to indicate that while there might be a "
#     "theoretical basis for their use in stroke treatment, practical application is limited by side effects such as "
#     "somnolence (drowsiness) and rhinitis (inflammation of the nasal mucous membrane). "
    
#     "User: What is total anterior circulation syndrome? "
#     "Assistant: Total Anterior Circulation Syndrome (TACS) is one of the classifications used to describe a "
#     "clinical pattern of stroke, specifically affecting the anterior part of the brain's blood supply. It is part "
#     "of the Oxford Community Stroke Project (OCSP) classification, which categorizes strokes based on the symptoms "
#     "they produce and the areas of the brain affected. TACS is associated with a blockage or reduced blood flow in "
#     "the main arteries supplying the front part of the brain, particularly the middle cerebral artery and sometimes "
#     "parts of the anterior cerebral artery. This type of stroke typically results in a more extensive area of brain "
#     "infarction. The classic symptoms of TACS include weakness or sensory loss on one side of the body, higher "
#     "cortical dysfunction such as speech problems or spatial neglect, and loss of half of the visual field in both "
#     "eyes on the same side. A diagnosis of TACS generally implies a large stroke with significant disability and a "
#     "poorer prognosis compared to other stroke subtypes. In research, understanding TACS is important because "
#     "treatment response and outcomes can differ by stroke subtype. "
    
#     "User: What is chlormethiazole? "
#     "Assistant: Chlormethiazole is a sedative and hypnotic drug that acts primarily as a GABA receptor agonist. By "
#     "activating GABA receptors, it enhances the brain’s main calming system. It has been used for sedation, "
#     "including in insomnia and alcohol withdrawal. It has also been studied for possible neuroprotective effects "
#     "in acute stroke. Its major side effects include somnolence (strong drowsiness) and rhinitis (nasal "
#     "inflammation), which limit its use, especially in stroke where too much sedation can be harmful."

#     "User: How is Medline involved in this study? "
#     "Assistant:"
#     "MEDLINE is involved in this study only as a literature source, not as part of the experiment itself."

#     "The authors were conducting a systematic review, so they needed to find all existing studies about GABA receptor agonists for acute stroke. To do this, they searched several medical databases, including MEDLINE, Embase, CENTRAL, and Chinese databases."

#     "MEDLINE’s role was to serve as a database of published medical research. The researchers used it to locate randomized controlled trials that studied GABA receptor agonists in patients with acute stroke and reported outcomes like death, disability, and side effects."

#     "In summary, MEDLINE did not produce new data for this study. It was used to find previously published studies that were then analyzed in the Cochrane Review."

#     "User: Explain Placebo?"
#     "Assistant: A placebo is something that looks like a real medical treatment but has no active medical ingredient or therapeutic effect."
#     "It’s often used in clinical trials to see whether a new drug or treatment actually works better than “doing nothing” or better than expectations alone."
# )

conversation_text = (
    "What are quality improvement (QI) programmes, and how do they help adults living with diabetes?\n"
    "What are some examples of QI strategies mentioned in the abstract?\n"
    "How do system-targeted, provider-targeted, and patient-targeted QI strategies differ from each other?\n"
    "What databases were searched to gather studies for this review, and what criteria were used to select studies?\n"
    "What outcomes were used to measure the effectiveness of QI strategies for people with diabetes?\n"
    "What is glycated haemoglobin (HbA1c), and why is it significant for diabetes management?\n"
    "How do multivariable meta-regression models in a Bayesian framework work, and why were they used in this analysis?\n"
    "What does it mean for QI strategies to be ranked within “Top,” “Middle,” and “Bottom” tiers?\n"
    "How did combinations of QI strategies impact glucose levels, blood pressure, and cholesterol in people with diabetes?\n"
    "Why might the evidence about QI strategies be limited, and how can this limitation affect decision-making for healthcare systems?\n"
)

sata_text = (
    "How do various quality improvement strategies compare in their effectiveness for improving blood sugar control in people with different baseline HbA1c levels?\n"
    "Choices: Strategies involving patient education and case management are most effective for those with higher baseline HbA1c., "
    "Clinician education is effective regardless of baseline HbA1c levels., "
    "There is no difference in effectiveness based on baseline HbA1c levels.\n"
    "Correct answer: Strategies involving patient education and case management are most effective for those with higher baseline HbA1c.\n\n"

    "Critique the role of patient education among the most effective strategies identified in the study.\n"
    "Choices: Patient education is essential but must be combined with other strategies for maximum efficacy., "
    "Patient education alone surpasses all other quality improvement strategies., "
    "Patient education is ineffective unless paired with team changes.\n"
    "Correct answer: Patient education is essential but must be combined with other strategies for maximum efficacy.\n\n"

    "What hypothesis can be generated about the relationship between multicomponent quality improvement programmes and patient-level outcomes?\n"
    "Choices: Multicomponent strategies may lead to population-level improvements in diabetic outcomes., "
    "Single-strategy programmes are equally effective as multicomponent ones., "
    "The effectiveness of multicomponent programmes diminishes with fewer strategies included.\n"
    "Correct answer: Multicomponent strategies may lead to population-level improvements in diabetic outcomes.\n\n"

    "Which strategies were part of the three most effective combinations for lowering systolic blood pressure?\n"
    "Choices: Case management, team changes, promotion of self-management, "
    "Clinician education, patient reminders, continuous quality improvement, "
    "Patient education, facilitated relay of clinical information, audit and feedback.\n"
    "Correct answer: Case management, team changes, promotion of self-management"
)

abstract = """There is a large body of evidence evaluating quality improvement (QI) programmes to improve care for adults living with diabetes. These programmes are often comprised of multiple QI strategies, which may be implemented in various combinations. Decision-makers planning to implement or evaluate a new QI programme, or both, need reliable evidence on the relative effectiveness of different QI strategies (individually and in combination) for different patient populations.
To update existing systematic reviews of diabetes QI programmes and apply novel meta-analytical techniques to estimate the effectiveness of QI strategies (individually and in combination) on diabetes quality of care.
We searched databases (CENTRAL, MEDLINE, Embase and CINAHL) and trials registers (ClinicalTrials.gov and WHO ICTRP) to 4 June 2019. We conducted a top-up search to 23 September 2021; we screened these search results and 42 studies meeting our eligibility criteria are available in the awaiting classification section.
We included randomised trials that assessed a QI programme to improve care in outpatient settings for people living with diabetes. QI programmes needed to evaluate at least one system- or provider-targeted QI strategy alone or in combination with a patient-targeted strategy. - System-targeted: case management (CM); team changes (TC); electronic patient registry (EPR); facilitated relay of clinical information (FR); continuous quality improvement (CQI). - Provider-targeted: audit and feedback (AF); clinician education (CE); clinician reminders (CR); financial incentives (FI). - Patient-targeted: patient education (PE); promotion of self-management (PSM); patient reminders (PR). Patient-targeted QI strategies needed to occur with a minimum of one provider or system-targeted strategy.
We dual-screened search results and abstracted data on study design, study population and QI strategies. We assessed the impact of the programmes on 13 measures of diabetes care, including: glycaemic control (e.g. mean glycated haemoglobin (HbA1c)); cardiovascular risk factor management (e.g. mean systolic blood pressure (SBP), low-density lipoprotein cholesterol (LDL-C), proportion of people living with diabetes that quit smoking or receiving cardiovascular medications); and screening/prevention of microvascular complications (e.g. proportion of patients receiving retinopathy or foot screening); and harms (e.g. proportion of patients experiencing adverse hypoglycaemia or hyperglycaemia). We modelled the association of each QI strategy with outcomes using a series of hierarchical multivariable meta-regression models in a Bayesian framework. The previous version of this review identified that different strategies were more or less effective depending on baseline levels of outcomes. To explore this further, we extended the main additive model for continuous outcomes (HbA1c, SBP and LDL-C) to include an interaction term between each strategy and average baseline risk for each study (baseline thresholds were based on a data-driven approach; we used the median of all baseline values reported in the trials). Based on model diagnostics, the baseline interaction models for HbA1c, SBP and LDL-C performed better than the main model and are therefore presented as the primary analyses for these outcomes. Based on the model results, we qualitatively ordered each QI strategy within three tiers (Top, Middle, Bottom) based on its magnitude of effect relative to the other QI strategies, where 'Top' indicates that the QI strategy was likely one of the most effective strategies for that specific outcome. Secondary analyses explored the sensitivity of results to choices in model specification and priors. Additional information about the methods and results of the review are available as Appendices in an online repository. This review will be maintained as a living systematic review; we will update our syntheses as more data become available.
We identified 553 trials (428 patient-randomised and 125 cluster-randomised trials), including a total of 412,161 participants. Of the included studies, 66% involved people living with type 2 diabetes only. Participants were 50% female and the median age of participants was 58.4 years. The mean duration of follow-up was 12.5 months. HbA1c was the commonest reported outcome; screening outcomes and outcomes related to cardiovascular medications, smoking and harms were reported infrequently. The most frequently evaluated QI strategies across all study arms were PE, PSM and CM, while the least frequently evaluated QI strategies included AF, FI and CQI. Our confidence in the evidence is limited due to a lack of information on how studies were conducted. Four QI strategies (CM, TC, PE, PSM) were consistently identified as 'Top' across the majority of outcomes. All QI strategies were ranked as 'Top' for at least one key outcome. The majority of effects of individual QI strategies were modest, but when used in combination could result in meaningful population-level improvements across the majority of outcomes. The median number of QI strategies in multicomponent QI programmes was three. Combinations of the three most effective QI strategies were estimated to lead to the below effects: - PR + PSM + CE: decrease in HbA1c by 0.41% (credibility interval (CrI) -0.61 to -0.22) when baseline HbA1c < 8.3%; - CM + PE + EPR: decrease in HbA1c by 0.62% (CrI -0.84 to -0.39) when baseline HbA1c > 8.3%; - PE + TC + PSM: reduction in SBP by 2.14 mmHg (CrI -3.80 to -0.52) when baseline SBP < 136 mmHg; - CM + TC + PSM: reduction in SBP by 4.39 mmHg (CrI -6.20 to -2.56) when baseline SBP > 136 mmHg; - TC + PE + CM: LDL-C lowering of 5.73 mg/dL (CrI -7.93 to -3.61) when baseline LDL < 107 mg/dL; - TC + CM + CR: LDL-C lowering by 5.52 mg/dL (CrI -9.24 to -1.89) when baseline LDL > 107 mg/dL. Assuming a baseline screening rate of 50%, the three most effective QI strategies were estimated to lead to an absolute improvement of 33% in retinopathy screening (PE + PR + TC) and 38% absolute increase in foot screening (PE + TC + Other).
There is a significant body of evidence about QI programmes to improve the management of diabetes. Multicomponent QI programmes for diabetes care (comprised of effective QI strategies) may achieve meaningful population-level improvements across the majority of outcomes. For health system decision-makers, the evidence summarised in this review can be used to identify strategies to include in QI programmes. For researchers, this synthesis identifies higher-priority QI strategies to examine in further research regarding how to optimise their evaluation and effects. We will maintain this as a living systematic review."""

In [11]:
def rewrite_abstract(client_openai, abstract: str, conversation_text: str, sata_text: str) -> str:
    sata_text = format_sata(sata_text)
    system_prompt = (
        "CRITICAL RULE:\n"
        "If the user asks 'why' or 'how' and the abstract does not explicitly explain the reason or mechanism, you MUST say so directly "
            "(e.g., 'The abstract does not explain why/how X; it only states that X was done.'). After this admission, you MUST provide a "
        "best-guess explanation using your internal background knowledge. Clearly label this as inference or general context (e.g., 'In a "
        "broader statistical context, this often means...') to distinguish it from the abstract's claims.\n\n"
        "You are an expert science communicator.\n\n"
        "Your task is to rewrite the abstract into a personalized, plain-language summary for this specific reader.\n\n"
        "You MUST use the questions the user is confused about below—specifically the query regarding the Bayesian framework—to ensure "
        "those concepts are demystified in the narrative.\n\n"
        f"Questions:\n{conversation_text}\n\n"
        f"Select-All-That-Apply (SATA) Questions:\n{sata_text}\n\n"
        "For each SATA item:\n"
        "- The rewritten summary MUST contain information that allows a careful reader to logically deduce every correct answer.\n"
        "- You must NOT explicitly list, label, or reference answer choices or say which options are correct inside the summary.\n"
        "- The summary MUST avoid adding statements that would also justify incorrect options.\n"
        "- You MUST include background knowledge for any technical term (like 'Bayesian' or 'HbA1c') mentioned in the user's questions, even if the abstract is silent on the details.\n"
        "- The summary must remain natural narrative, not exam-style reasoning.\n\n"
        "Final output rules:\n"
        "- Output only the final personalized plain-language summary.\n"
        "- The summary MUST be written in coherent paragraph form.\n"
        "- Do NOT use bullet points, lists, headings, or numbered items.\n"
        "- LENGTH CONSTRAINT: The rewritten summary MUST have approximately the same number of sentences as the original abstract (±1 sentence).\n"
    )

    example_abstract = (
        "Exercise is generally recommended for people with type 2 diabetes mellitus. However, some studies "
        "evaluate an exercise intervention including diet or behaviour modification or both, and the effects "
        "of diet and exercise are not differentiated. Some exercise studies involve low participant numbers, "
        "lacking power to show significant differences which may appear in larger trials. "
        "To assess the effects of exercise in type 2 diabetes mellitus. "
        "Trials were identified through the Cochrane Central Register of Controlled Trials (CENTRAL), MEDLINE, "
        "EMBASE and manual searches of bibliographies. "
        "All randomised controlled trials comparing any type of well-documented aerobic, fitness or progressive "
        "resistance training exercise with no exercise in people with type 2 diabetes mellitus. "
        "Two authors independently selected trials, assessed trial quality and extracted data. Study authors "
        "were contacted for additional information. Any information on adverse effects was collected from the trials. "
        "Fourteen randomised controlled trials comparing exercise against no exercise in type 2 diabetes were "
        "identified involving 377 participants. Trials ranged from eight weeks to twelve months duration. "
        "Compared with the control, the exercise intervention significantly improved glycaemic control as "
        "indicated by a decrease in glycated haemoglobin levels of 0.6%. "
        "There was no significant difference between groups in whole body mass, probably due to an increase "
        "in fat free mass (muscle) with exercise. "
        "There was a reduction in visceral adipose tissue with exercise. "
        "No study reported adverse effects in the exercise group or diabetic complications."
    )

    example_conversation = (
        "What is type 2 diabetes mellitus?; "
        "Why is exercise generally recommended for people with type 2 diabetes?; "
        "What types of exercise were included in the trials mentioned?; "
        "How does exercise improve glycaemic control in people with type 2 diabetes?;"
    )

    example_sata = (
        "How might researchers justify the clinical relevance of the findings despite no noted weight loss in participants?\n"
        "Choices: Improvements in insulin response and decreased body fat, Lack of adverse effects from exercise interventions, "
        "Extended duration of exercise interventions\n"
        "Correct answers: Improvements in insulin response and decreased body fat\n\n"
        "What was the main aim of the review authors?\n"
        "Choices: To determine the effect of exercise on blood sugar control in type 2 diabetes., "
        "To compare dietary and exercise interventions., To analyze medication effects on type 2 diabetes.\n"
        "Correct answers: To determine the effect of exercise on blood sugar control in type 2 diabetes.\n\n"
        "Which outcome did exercise improve in type 2 diabetes management?\n"
        "Choices: Blood sugar control, Blood cholestrol levels, Blood pressure\n"
        "Correct answer: Blood sugar control\n\n"
        "What was a significant effect of exercise identified in the trials?\n"
        "Choices: Decreased body fat content, Increased overall body weight, Higher energy expenditure\n"
        "Correct answer: Decreased body fat content"
    )

    example_output = (
        "Type 2 diabetes mellitus is a condition where the body has trouble regulating blood sugar levels. "
        "For managing this, exercise is often recommended because it helps improve how the body controls blood sugar. "
        "In this analysis, researchers looked at studies to understand the effects of exercise alone on people with "
        "type 2 diabetes. These studies included different types of exercises, such as aerobic exercises, fitness "
        "programs, and resistance training.\n\n"
        "They found that exercise helped lower blood sugar levels, as shown by a decrease in glycated haemoglobin (HbA1c). "
        "Glycated haemoglobin is important in diabetes management because it measures average blood sugar over time. "
        "Exercise did not lead to overall weight loss, but it did increase muscle mass and reduce harmful abdominal fat.\n\n"
        "Improved insulin response was also noted, and no adverse effects were reported. Together, these findings show "
        "that exercise can improve blood sugar control and body composition even without weight loss."
    )


    example_grounding = (
        "DEMONSTRATION — QUESTION COVERAGE AND GROUNDING (NOT PART OF FINAL OUTPUT)\n"
        "IMPORTANT: All sentences referenced below are taken verbatim from the rewritten abstract above. "
        "No new information is added. This section exists only to demonstrate coverage.\n\n"

        "LAY QUESTIONS:\n"
        "What is type 2 diabetes mellitus? → "
        "\"Type 2 diabetes mellitus is a condition where the body has trouble regulating blood sugar levels.\"\n"
        "Why is exercise generally recommended for people with type 2 diabetes? → "
        "\"exercise is often recommended because it helps improve how the body controls blood sugar.\"\n"
        "What types of exercise were included in the trials mentioned? → "
        "\"aerobic exercises, fitness programs, and resistance training.\"\n"
        "How does exercise improve glycaemic control in people with type 2 diabetes? → "
        "\"exercise helped lower blood sugar levels, as shown by a decrease in glycated haemoglobin (HbA1c).\"\n\n"

        "SATA QUESTIONS:\n"
        "SATA Question:\n"
        "How might researchers justify the clinical relevance of the findings despite no noted weight loss in participants?\n"
        "Evidence sentence(s) from the rewritten abstract enabling deduction:\n"
        "\"Exercise did not lead to overall weight loss, but it did increase muscle mass and reduce harmful abdominal fat.\"\n"
        "\"Improved insulin response was also noted.\"\n\n"

        "SATA Question:\n"
        "What was the main aim of the review authors?\n"
        "Evidence sentence(s) from the rewritten abstract enabling deduction:\n"
        "\"In this analysis, researchers looked at studies to understand the effects of exercise alone on people with type 2 diabetes.\"\n"
        "\"They found that exercise helped lower blood sugar levels.\"\n\n"

        "SATA Question:\n"
        "Which outcome did exercise improve in type 2 diabetes management?\n"
        "Evidence sentence(s) from the rewritten abstract enabling deduction:\n"
        "\"They found that exercise helped lower blood sugar levels, as shown by a decrease in glycated haemoglobin (HbA1c).\"\n\n"

        "SATA Question:\n"
        "What was a significant effect of exercise identified in the trials?\n"
        "Evidence sentence(s) from the rewritten abstract enabling deduction:\n"
        "\"Exercise did not lead to overall weight loss, but it did increase muscle mass and reduce harmful abdominal fat.\"\n"
    )

    response = client_openai.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": (
                        f"Rewrite this abstract:\n\n{example_abstract}\n\n"
                        f"Questions:\n{example_conversation}\n\n"
                        f"SATA:\n{example_sata}\n\n"
                        f"---\n"
                        f"Below is an EXAMPLE of a correct rewritten summary, followed by an "
                        f"EXAMPLE-ONLY grounding that shows how the rewritten text answers all questions.\n"
                        f"The grounding is NOT part of the desired output.\n\n"
                        f"REWRITTEN SUMMARY (EXAMPLE):\n{example_output}\n\n"
                        f"GROUNDING (EXAMPLE ONLY):\n{example_grounding}"
                    )
                },

            # Actual task
            {"role": "user", "content": f"Rewrite this abstract:\n\n{abstract}"},
        ],
    )
    return response.choices[0].message.content.strip()

In [12]:
# personalized_summary = rewrite_abstract(client_openai, abstract, conversation_text, sata_text)
# print(personalized_summary)

In [13]:
def generate_lay_questions_abstract(client_openai, abstract: str) -> str:
    system_prompt = (
        "You are a layperson with no scientific or medical background."
        "Your task is to generate up to 5 questions that you would need answered"
        "in order to understand the following abstract. "
        "Output the questions as a single line, separated by semicolons."
    )

    response = client_openai.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": f"Abstract:\n{abstract}\n\n"
                           "Please list up to 5 questions, separated by semicolons."
            },
        ],
    )
    print(response.choices[0].message.content.strip())
    return response.choices[0].message.content.strip()

In [14]:
abstract_df = pd.read_csv("new_abstract_dataset.csv")
abstract_df.loc[:10, "lay_person_question"] = (abstract_df.loc[:10, "abstract"].apply(lambda x: generate_lay_questions_abstract(client_openai, x)))
abstract_df.to_csv("new_abstract_dataset.csv")

What exactly counts as “exercise” in these trials (type, intensity, how often, how long), and does one kind work better than others?; What does a 0.6% drop in HbA1c mean in everyday terms for someone with type 2 diabetes (how big/important is that change)?; If body weight didn’t change, how can exercise still improve blood sugar—what do “fat free mass” and “visceral adipose tissue” mean and why do they matter?; How reliable are these results given there were only 377 people across 14 studies—were the studies good quality and long enough to be confident?; Were there any risks or downsides of exercising for these participants, and why might no adverse effects have been reported?
What exactly counts as a “quality improvement (QI) strategy” in this review, and what do examples like case management, team changes, and patient self-management promotion look like in real clinics?; How should I interpret the reported changes (e.g., HbA1c down 0.62%, systolic blood pressure down 4.39 mmHg, LDL d

In [15]:
import ast

def build_sata_from_row(row):
    chosen = row["chosen_questions"]
    if isinstance(chosen, str):
        chosen = ast.literal_eval(chosen)

    sata_text = []

    for q_num in chosen:
        q_key = f"q{q_num}"

        question = row[f"{q_key}_question"]

        options = []
        correct_answers = []

        for opt_i in [1, 2, 3]:
            opt_text = row[f"{q_key}_option{opt_i}_answer"]
            opt_correct = row[f"{q_key}_option{opt_i}_correct"]

            if pd.isna(opt_text):
                continue

            options.append(opt_text)
            if opt_correct is True or str(opt_correct).upper() == "TRUE":
                correct_answers.append(opt_text)

        sata_text.append({
            "question": question,
            "options": options,
            "correct_answers": correct_answers
        })

    return sata_text

In [16]:
abstract_ids = ["1001", "1002", "1003", "1004", "1005", "1006", "1007", "1008", "1009", "1011"]
abstract_ids_set = set(abstract_ids)

abstract_df["generated_summary"] = None

for idx, row in abstract_df.iterrows():
    if str(row["abstract_id"]) not in abstract_ids_set:
        continue

    conversation_text = "; ".join(
        q.strip()
        for q in row["lay_person_question"].split(";")
        if q.strip()
    ) + ";"

    sata_text = build_sata_from_row(row)
    generated = rewrite_abstract(
        client_openai,
        abstract=row["abstract"],
        conversation_text=conversation_text,
        sata_text=sata_text
    )

    abstract_df.at[idx, "generated_summary"] = generated

abstract_df.to_csv("new_abstract_dataset.csv")